In [1]:
!pip install datasets transformers evaluate sacrebleu bert_score rouge-score gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 76.1 MB/s eta 0:00:00
 

In [4]:
import torch
import gradio as gr
from datasets import load_dataset, DatasetDict
from transformers import BartTokenizer
from transformers import BartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from evaluate import load

In [3]:
# Load dataset
dataset = load_dataset("multi_news")

# Train/validation split
split = dataset["train"].train_test_split(test_size=0.1, seed=42)
dataset = DatasetDict({
    "train": split["train"],
    "validation": split["test"],
    "test": dataset["validation"]
})

# Load tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Preprocessing function
def preprocess(example):
    model_input = tokenizer(example["document"], max_length=1024, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["summary"], max_length=256, truncation=True, padding="max_length")
    model_input["labels"] = labels["input_ids"]
    return model_input

# Tokenize
tokenized = dataset.map(preprocess, batched=True, remove_columns=["document", "summary"])
tokenized.save_to_disk("tokenized_multinews")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

multi_news.py:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train.src.cleaned:   0%|          | 0.00/548M [00:00<?, ?B/s]

...

Map:   0%|          | 0/4498 [00:00<?, ? examples/s]

Map:   0%|          | 0/5622 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/40474 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4498 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5622 [00:00<?, ? examples/s]

In [7]:
# Load model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to("cuda")

rouge = load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    labels = [[tokenizer.decode(l, skip_special_tokens=True)] for l in labels]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=[l[0] for l in labels])
    return {k: round(v*100, 2) for k, v in result.items()}

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model)

# Training arguments
args = Seq2SeqTrainingArguments(
    output_dir="./results_multinews",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    logging_dir="./logs_multinews",
    report_to="none"
)


# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"].select(range(1000)),
    eval_dataset=tokenized["validation"].select(range(100)),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-26f7edc889d5>:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,3.059700,2.401961,9.800000,3.190000,7.380000,7.410000
2,2.510900,2.368339,9.770000,2.990000,7.320000,7.330000
3,2.386400,2.358691,10.040000,3.260000,7.480000,7.500000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1500, training_loss=2.6523271484375, metrics={'train_runtime': 348.6573, 'train_samples_per_second': 8.604, 'train_steps_per_second': 4.302, 'total_flos': 1829209374720000.0, 'train_loss': 2.6523271484375, 'epoch': 3.0})

In [8]:
# Metrics
bleu = load("sacrebleu")
bert = load("bertscore")

test_set = tokenized["test"].select(range(100))
labels = []
preds = []

model.eval()
for sample in test_set:
    inputs = torch.tensor(sample["input_ids"]).unsqueeze(0).to("cuda")
    mask = torch.tensor(sample["attention_mask"]).unsqueeze(0).to("cuda")
    output = model.generate(inputs, attention_mask=mask, max_length=256)
    labels.append(tokenizer.decode(sample["labels"], skip_special_tokens=True))
    preds.append(tokenizer.decode(output[0], skip_special_tokens=True))



rouge_scores = rouge.compute(predictions=preds, references=labels)
bleu_score = bleu.compute(predictions=preds, references=[[r] for r in labels])
bert_score = bert.compute(predictions=preds, references=labels, lang="en")

print("ROUGE:", {k: round(v * 100, 2) for k, v in rouge_scores.items()})
print("BLEU Score:", round(bleu_score["score"], 2))
print("BERTScore F1:", round(sum(bert_score["f1"]) / len(bert_score["f1"]) * 100, 2))


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ROUGE: {'rouge1': np.float64(40.11), 'rouge2': np.float64(13.4), 'rougeL': np.float64(20.79), 'rougeLsum': np.float64(20.86)}
BLEU Score: 11.24
BERTScore F1: 85.76


In [9]:
def summarize_news(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=1024).to("cuda")
    output = model.generate(**inputs, max_length=256)
    return tokenizer.decode(output[0], skip_special_tokens=True)

gr.Interface(fn=summarize_news, inputs="textbox", outputs="textbox", title="📰 Multi-News Summarizer").launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://500f930e782c3d9f5d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**References:**
* https://huggingface.co/datasets/multi_news
* https://huggingface.co/facebook/bart-base
* https://huggingface.co/docs/transformers/model_doc/bart
* https://huggingface.co/docs/evaluate/index
* https://www.gradio.app


**UB box link to Video Recording:**

https://buffalo.box.com/s/qxc3ain1f6ic16nwaydd136381z056db